# STGCN代码复现
* [源仓库地址](https://github.com/hazdzz/STGCN)

### 导入python库和文件

In [1]:
import logging
import os
import argparse
import math
import random
import tqdm
import numpy as np
import pandas as pd
from sklearn import preprocessing

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils

from script import dataloader, utility, earlystopping
from model import models

from param_parser import parameter_parser, tab_printer


In [2]:
def set_env(seed):
    # Set available CUDA devices
    # This option is crucial for an multi-GPU device
    os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'
    # os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
    # os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':16:8'
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
    os.environ['PYTHONHASHSEED']=str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms(True)

In [3]:
args = parameter_parser()

In [4]:
tab_printer(args)

+-------------------+-----------------+
|     Parameter     |      Value      |
+===================+=================+
| Ks                | 3               |
+-------------------+-----------------+
| Kt                | 3               |
+-------------------+-----------------+
| Act func          | glu             |
+-------------------+-----------------+
| Batch size        | 32              |
+-------------------+-----------------+
| Dataset           | metr-la         |
+-------------------+-----------------+
| Droprate          | 0.500           |
+-------------------+-----------------+
| Enable bias       | 1               |
+-------------------+-----------------+
| Enable cuda       | 1               |
+-------------------+-----------------+
| Epochs            | 10000           |
+-------------------+-----------------+
| Gamma             | 0.950           |
+-------------------+-----------------+
| Graph conv type   | cheb_graph_conv |
+-------------------+-----------------+


In [5]:
logging.basicConfig(level=logging.INFO)

In [6]:
 # For stable experiment results
set_env(args.seed)

In [7]:
# Running in Nvidia GPU (CUDA) or CPU
if args.enable_cuda and torch.cuda.is_available():
    # Set available CUDA devices
    # This option is crucial for multiple GPUs
    # 'cuda' ≡ 'cuda:0'
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cpu


In [8]:
Ko = args.n_his - (args.Kt - 1) * 2 * args.stblock_num

# blocks: settings of channel size in st_conv_blocks and output layer,
# using the bottleneck design in st_conv_blocks
blocks = []
blocks.append([1])
for l in range(args.stblock_num):
    blocks.append([64, 16, 64])
if Ko == 0:
    blocks.append([128])
elif Ko > 0:
    blocks.append([128, 128])

blocks.append([1])

print(blocks)

[[1], [64, 16, 64], [64, 16, 64], [128, 128], [1]]


### 数据准备

In [9]:
def data_preparate(args, device):    
    adj, n_vertex = dataloader.load_adj(args.dataset)
    gso = utility.calc_gso(adj, args.gso_type)
    if args.graph_conv_type == 'cheb_graph_conv':
        gso = utility.calc_chebynet_gso(gso)
    gso = gso.toarray()
    gso = gso.astype(dtype=np.float32)
    args.gso = torch.from_numpy(gso).to(device)

    dataset_path = './data'
    dataset_path = os.path.join(dataset_path, args.dataset)
    data_col = pd.read_csv(os.path.join(dataset_path, 'vel.csv')).shape[0]
    # recommended dataset split rate as train: val: test = 60: 20: 20, 70: 15: 15 or 80: 10: 10
    # using dataset split rate as train: val: test = 70: 15: 15
    val_and_test_rate = 0.15

    len_val = int(math.floor(data_col * val_and_test_rate))
    len_test = int(math.floor(data_col * val_and_test_rate))
    len_train = int(data_col - len_val - len_test)

    train, val, test = dataloader.load_data(args.dataset, len_train, len_val)
    zscore = preprocessing.StandardScaler()
    train = zscore.fit_transform(train)
    val = zscore.transform(val)
    test = zscore.transform(test)

    x_train, y_train = dataloader.data_transform(train, args.n_his, args.n_pred, device)
    x_val, y_val = dataloader.data_transform(val, args.n_his, args.n_pred, device)
    x_test, y_test = dataloader.data_transform(test, args.n_his, args.n_pred, device)

    train_data = utils.data.TensorDataset(x_train, y_train)
    train_iter = utils.data.DataLoader(dataset=train_data, batch_size=args.batch_size, shuffle=False)
    val_data = utils.data.TensorDataset(x_val, y_val)
    val_iter = utils.data.DataLoader(dataset=val_data, batch_size=args.batch_size, shuffle=False)
    test_data = utils.data.TensorDataset(x_test, y_test)
    test_iter = utils.data.DataLoader(dataset=test_data, batch_size=args.batch_size, shuffle=False)

    return n_vertex, zscore, train_iter, val_iter, test_iter

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
n_vertex, zscore, train_iter, val_iter, test_iter = data_preparate(args, device)

### 模型准备

In [12]:
def prepare_model(args, blocks, n_vertex):
    loss = nn.MSELoss()
    es = earlystopping.EarlyStopping(mode='min', min_delta=0.0, patience=args.patience)

    if args.graph_conv_type == 'cheb_graph_conv':
        model = models.STGCNChebGraphConv(args, blocks, n_vertex).to(device)
    else:
        model = models.STGCNGraphConv(args, blocks, n_vertex).to(device)

    if args.opt == "rmsprop":
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.weight_decay_rate)
    elif args.opt == "adam":
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay_rate, amsgrad=False)
    elif args.opt == "adamw":
        optimizer = optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay_rate, amsgrad=False)
    else:
        raise NotImplementedError(f'ERROR: The optimizer {args.opt} is not implemented.')

    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)

    return loss, es, model, optimizer, scheduler

In [13]:
loss, es, model, optimizer, scheduler = prepare_model(args, blocks, n_vertex)

### 模型训练

In [14]:
@torch.no_grad()
def val(model, val_iter):
    model.eval()
    l_sum, n = 0.0, 0
    for x, y in val_iter:
        y_pred = model(x).view(len(x), -1)
        l = loss(y_pred, y)
        l_sum += l.item() * y.shape[0]
        n += y.shape[0]
    return torch.tensor(l_sum / n)

In [15]:
def train(loss, args, optimizer, scheduler, es, model, train_iter, val_iter):
    for epoch in range(args.epochs):
        l_sum, n = 0.0, 0  # 'l_sum' is epoch sum loss, 'n' is epoch instance number
        model.train()
        for x, y in tqdm.tqdm(train_iter):
            y_pred = model(x).view(len(x), -1)  # [batch_size, num_nodes]
            l = loss(y_pred, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        scheduler.step()
        val_loss = val(model, val_iter)
        # GPU memory usage
        gpu_mem_alloc = torch.cuda.max_memory_allocated() / 1000000 if torch.cuda.is_available() else 0
        print('Epoch: {:03d} | Lr: {:.20f} |Train loss: {:.6f} | Val loss: {:.6f} | GPU occupy: {:.6f} MiB'.\
            format(epoch+1, optimizer.param_groups[0]['lr'], l_sum / n, val_loss, gpu_mem_alloc))

        if es.step(val_loss):
            print('Early stopping.')
            break

In [16]:
args.epochs = 3
train(loss, args, optimizer, scheduler, es, model, train_iter, val_iter)

100%|██████████| 750/750 [09:53<00:00,  1.26it/s]


Epoch: 001 | Lr: 0.00100000000000000002 |Train loss: 0.351105 | Val loss: 0.453681 | GPU occupy: 0.000000 MiB


100%|██████████| 750/750 [09:11<00:00,  1.36it/s]


Epoch: 002 | Lr: 0.00100000000000000002 |Train loss: 0.282300 | Val loss: 0.344895 | GPU occupy: 0.000000 MiB


100%|██████████| 750/750 [09:31<00:00,  1.31it/s]


Epoch: 003 | Lr: 0.00100000000000000002 |Train loss: 0.275790 | Val loss: 0.354051 | GPU occupy: 0.000000 MiB


### 模型测试

In [17]:
@torch.no_grad() 
def test(zscore, loss, model, test_iter, args):
    model.eval()
    test_MSE = utility.evaluate_model(model, loss, test_iter)
    test_MAE, test_RMSE, test_WMAPE = utility.evaluate_metric(model, test_iter, zscore)
    print(f'Dataset {args.dataset:s} | Test loss {test_MSE:.6f} | MAE {test_MAE:.6f} | RMSE {test_RMSE:.6f} | WMAPE {test_WMAPE:.8f}')

In [18]:
test(zscore, loss, model, test_iter, args)

Dataset metr-la | Test loss 0.408878 | MAE 6.813248 | RMSE 11.466577 | WMAPE 0.13412205
